In [1]:
!pip install transformers[ja]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 7.2 MB/s 
     |████████████████████████████████| 7.6 MB 51.8 MB/s 
     |████████████████████████████████| 163 kB 64.5 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 2.2 MB 59.6 MB/s 
     |████████████████████████████████| 583 kB 61.3 MB/s 
     |████████████████████████████████| 13.4 MB 55.5 MB/s 
     |████████████████████████████████| 47.4 MB 124 kB/s 
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556723 sha256=9b548e3bf9cb48b150f5e9bc34ffa256973521e64f3e9239ffdeed18fc5eb1ff
  Stored in directory: /root/.cache/pip/wheels/33/8b/99/cf0d27191876637cd3639a560f93aa982d7855ce826c94348b
  Created wheel for sudachidict-core: filename=SudachiDict_core-20220729-py3-none-any.whl size=71570387 sha256=61dfbd2c5975bd4b838ef14239650b6b1cf28d73d7344948c5980e362f18c131
  S

In [2]:
!pip list | grep torch
!pip list | grep transformers
!pip list | grep fugashi  
!pip list | grep ipadic

torch                         1.12.1+cu113
torchaudio                    0.12.1+cu113
torchsummary                  1.5.1
torchtext                     0.13.1
torchvision                   0.13.1+cu113
transformers                  4.23.1
fugashi                       1.2.0
ipadic                        1.0.0


In [3]:
!pip install torchtext==0.9.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.1 MB 18.5 MB/s 
     |████████████████████████████████| 804.1 MB 2.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.13.1
    Uninstalling torchtext-0.13.1:
      Successfully uninstalled torchtext-0.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.
torchaudio 0.12.1+cu113 requires torch==1.12.1, but you have torch 1.8.1 which is incompatible.


In [4]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext

# データ読み込み
# with open('livedoor_data.pickle', 'rb') as r:
#     livedoor_data = pickle.load(r)
livedoor_data = pd.read_table("all.tsv")

# データ確認
display(livedoor_data.head())

# 正解ラベル（カテゴリー）をデータセットから取得
categories = list(set(livedoor_data['category']))
print(categories)

# カテゴリーのID辞書を作成
id2cat = dict(zip(list(range(len(categories))), categories))
cat2id = dict(zip(categories, list(range(len(categories)))))
print(id2cat)
print(cat2id)

# DataFrameにカテゴリーID列を追加
livedoor_data['category_id'] = livedoor_data['category'].map(cat2id)

# 念の為シャッフル
livedoor_data = livedoor_data.sample(frac=1).reset_index(drop=True)

# データセットを本文とカテゴリーID列だけにする
livedoor_data = livedoor_data[['body', 'category_id']]
display(livedoor_data.head())

,category,labels,no,body
0,fact,1,NaN,韓国軍合同参謀本部は19日、北朝鮮が18日夜から19日午後にかけて、南北軍事境界線の近くで、...
1,fact,1,NaN,2018年の南北軍事合意で砲撃が禁じられた区域に着弾したが、韓国領海への着弾はなかった。
2,fact,1,NaN,これに対し、北朝鮮の朝鮮人民軍総参謀部の報道官は19日、砲撃の理由について、「敵（韓国軍）は...
3,fact,1,NaN,神奈川県弁護士会は１８日、刑務所で被収容者に閉居罰の受罰姿勢として正座やあぐらを強制すること...
4,fact,1,NaN,その日のうちに、渡辺容疑者を、被害男性に対する監禁の疑いで、緊急逮捕した。


['opinion', 'fact']
{0: 'opinion', 1: 'fact'}
{'opinion': 0, 'fact': 1}


,body,category_id
0,生前にリハーサルをすれば、本人確認のうえで、より理想に近づけられます。,0
1,たとえば企業の俊敏性向上にはトップから現場まで、デジタル時代に即した行動やコミュニケーション...,0
2,睡眠の質向上、ストレス緩和などの商品特性が消費者に浸透したからだ。,0
3,確かに二〇一三年以降の大規模金融緩和は雇用と株価を支えた。,1
4,週明け24日頃からは再び冷たい空気が流れ込み、最高気温、最低気温ともに平年を下回る所が多くな...,0


In [5]:
train_df, test_df = train_test_split(livedoor_data, train_size=0.8)
print("学習データサイズ", train_df.shape[0])
print("テストデータサイズ", test_df.shape[0])
#学習データサイズ 5900
#テストデータサイズ 1476

# tsvファイルとして保存する
train_df.to_csv('./train.tsv', sep='\t', index=False, header=None)
test_df.to_csv('./test.tsv', sep='\t', index=False, header=None)

学習データサイズ 480
テストデータサイズ 121


In [6]:
# !pip install transformers==4.5.1

In [7]:
# from transformers.modeling_bert import BertModel
from transformers import BertModel
# from transformers.tokenization_bert_japanese import BertJapaneseTokenizer
from transformers import BertJapaneseTokenizer

In [8]:
from transformers import BertModel
# from transformers import BertTokenizer
from transformers import AutoTokenizer
from torchtext.legacy import data

tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# BERTはサブワードを含めて最大512単語まで扱える
MAX_LENGTH = 512
def bert_tokenizer(text):
    return tokenizer.encode(text, max_length=MAX_LENGTH, truncation=True, return_tensors='pt')[0]

TEXT = torchtext.legacy.data.Field(sequential=True, tokenize=bert_tokenizer, use_vocab=False, lower=False,include_lengths=True, batch_first=True, fix_length=MAX_LENGTH, pad_token=0)
LABEL = torchtext.legacy.data.Field(sequential=False, use_vocab=False)

train_data, test_data = torchtext.legacy.data.TabularDataset.splits(path='./', train='train.tsv', test='test.tsv', format='tsv', fields=[('Text', TEXT), ('Label', LABEL)])

# BERTではミニバッチサイズは16か32が推奨される
BATCH_SIZE = 16
train_iter, test_iter = torchtext.legacy.data.Iterator.splits((train_data, test_data), batch_sizes=(BATCH_SIZE, BATCH_SIZE), repeat=False, sort=False)

Downloading:   0%|          | 0.00/110 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/258k [00:00<?, ?B/s]

In [9]:
from transformers import BertTokenizer
from transformers import AutoTokenizer

a_tokenizer = AutoTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')
b_tokenizer = BertTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

text = "人工知能は人間の仕事を奪った。"

ids = a_tokenizer.encode(text, return_tensors='pt')[0]
wakati = a_tokenizer.convert_ids_to_tokens(ids)
print(ids)
print(wakati)

ids = b_tokenizer.encode(text, return_tensors='pt')[0]
wakati = b_tokenizer.convert_ids_to_tokens(ids)
print(ids)
print(wakati)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'BertTokenizer'.


tensor([    2,  4969, 12588,     9,  1410,     5,  2198,    11, 11224,    10,
            8,     3])
['[CLS]', '人工', '知能', 'は', '人間', 'の', '仕事', 'を', '奪っ', 'た', '。', '[SEP]']
tensor([    2,    53,   461,   357,  1329,     9,    53,   284,     5,   757,
          146,    11,  1847,  4046, 28447,     8,     3])
['[CLS]', '人', '工', '知', '能', 'は', '人', '間', 'の', '仕', '事', 'を', '奪', 'っ', '##た', '。', '[SEP]']


In [10]:
class BertClassifier(nn.Module):
    def __init__(self):
        super(BertClassifier, self).__init__()

        # 日本語学習済モデルをロードする
        # output_attentions=Trueで順伝播のときにattention weightを受け取れるようにする
        # output_hidden_state=Trueで12層のBertLayerの隠れ層を取得する
        self.bert = BertModel.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking',
                                              output_attentions=True,
                                              output_hidden_states=True)

        # BERTの隠れ層の次元数は768だが、最終4層分のベクトルを結合したものを扱うので、７６８×4次元としている。
        self.linear = nn.Linear(768*4, 9)

        # 重み初期化処理
        nn.init.normal_(self.linear.weight, std=0.02)
        nn.init.normal_(self.linear.bias, 0)

    # clsトークンのベクトルを取得する用の関数を用意
    def _get_cls_vec(self, vec):
        return vec[:,0,:].view(-1, 768)

    def forward(self, input_ids):

        # 順伝播の出力結果は辞書形式なので、必要な値のkeyを指定して取得する
        output = self.bert(input_ids)
        attentions = output['attentions']
        hidden_states = output['hidden_states']

        # 最終４層の隠れ層からそれぞれclsトークンのベクトルを取得する
        vec1 = self._get_cls_vec(hidden_states[-1])
        vec2 = self._get_cls_vec(hidden_states[-2])
        vec3 = self._get_cls_vec(hidden_states[-3])
        vec4 = self._get_cls_vec(hidden_states[-4])

        # 4つのclsトークンを結合して１つのベクトルにする。
        vec = torch.cat([vec1, vec2, vec3, vec4], dim=1)

        # 全結合層でクラス分類用に次元を変換
        out = self.linear(vec)

        return F.log_softmax(out, dim=1), attentions

classifier = BertClassifier()

Downloading:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# まずは全部OFF
for param in classifier.parameters():
    param.requires_grad = False

# BERTの最終4層分をON
for param in classifier.bert.encoder.layer[-1].parameters():
    param.requires_grad = True

for param in classifier.bert.encoder.layer[-2].parameters():
    param.requires_grad = True

for param in classifier.bert.encoder.layer[-3].parameters():
    param.requires_grad = True

for param in classifier.bert.encoder.layer[-4].parameters():
    param.requires_grad = True

# クラス分類のところもON
for param in classifier.linear.parameters():
    param.requires_grad = True

# 事前学習済の箇所は学習率小さめ、最後の全結合層は大きめにする。
optimizer = optim.Adam([
    {'params': classifier.bert.encoder.layer[-1].parameters(), 'lr': 5e-5},
    {'params': classifier.bert.encoder.layer[-2].parameters(), 'lr': 5e-5},
    {'params': classifier.bert.encoder.layer[-3].parameters(), 'lr': 5e-5},
    {'params': classifier.bert.encoder.layer[-4].parameters(), 'lr': 5e-5},
    {'params': classifier.linear.parameters(), 'lr': 1e-4}
])

In [12]:
# 損失関数の設定
loss_function = nn.NLLLoss()

# GPUの設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# ネットワークをGPUへ送る
classifier.to(device)
losses = []

# エポック数は5で
for epoch in range(5):

    all_loss = 0

    for idx, batch in enumerate(train_iter):

        classifier.zero_grad()

        input_ids = batch.Text[0].to(device)
        label_ids = batch.Label.to(device)

        out, _ = classifier(input_ids)

        batch_loss = loss_function(out, label_ids)
        batch_loss.backward()

        optimizer.step()

        all_loss += batch_loss.item()

    print("epoch", epoch, "\t" , "loss", all_loss)

epoch 0 	 loss 24.331868410110474
epoch 1 	 loss 18.46911120414734
epoch 2 	 loss 15.598559319972992
epoch 3 	 loss 13.538590550422668
epoch 4 	 loss 11.974037319421768


In [13]:
answer = []
prediction = []

with torch.no_grad():
    for batch in test_iter:

        text_tensor = batch.Text[0].to(device)
        label_tensor = batch.Label.to(device)

        score, _ = classifier(text_tensor)
        _, pred = torch.max(score, 1)

        prediction += list(pred.cpu().numpy())
        answer += list(label_tensor.cpu().numpy())

print(classification_report(prediction, answer, target_names=categories))

              precision    recall  f1-score   support

     opinion       0.79      0.85      0.82        54
        fact       0.87      0.82      0.85        67

    accuracy                           0.83       121
   macro avg       0.83      0.84      0.83       121
weighted avg       0.84      0.83      0.84       121



In [14]:
def highlight(word, attn):
    html_color = '#%02X%02X%02X' % (255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}">{}</span>'.format(html_color, word)

def mk_html(index, batch, preds, attention_weight):
    sentence = batch.Text[0][index]
    label =batch.Label[index].item()
    pred = preds[index].item()

    label_str = id2cat[label]
    pred_str = id2cat[pred]

    html = "正解カテゴリ: {}<br>予測カテゴリ: {}<br>".format(label_str, pred_str)

    # 文章の長さ分のzero tensorを宣言
    seq_len = attention_weight.size()[2]
    all_attens = torch.zeros(seq_len).to(device)

    for i in range(12):
        all_attens += attention_weight[index, i, 0, :]

    for word, attn in zip(sentence, all_attens):
        if tokenizer.convert_ids_to_tokens([word.tolist()])[0] == "[SEP]":
            break
        html += highlight(tokenizer.convert_ids_to_tokens([word.numpy().tolist()])[0], attn) + " "
    html += "<br><br>"
    return html

batch = next(iter(test_iter))
score, attentions = classifier(batch.Text[0].to(device))
_, pred = torch.max(score, 1)

from IPython.display import display, HTML
for i in range(BATCH_SIZE):
    html_output = mk_html(i, batch, pred, attentions[-1])
    display(HTML(html_output))

In [19]:
# torch.save(classifier, 'model_weight.pth')
model_path = './model.pth'
torch.save(classifier.to('cpu').state_dict(), model_path)

In [ ]:
model_path = './model.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))